In [1]:
%pip install onnxruntime

%pip install -U scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U git+https://github.com/albumentations-team/albumentations

  Cloning https://github.com/albumentations-team/albumentations to c:\users\feers\appdata\local\temp\pip-req-build-d14quvu4
  Resolved https://github.com/albumentations-team/albumentations to commit f5ae3f5a9d8b77182c8a726deeadfc5dd1e93b8c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/albumentations-team/albumentations 'C:\Users\feers\AppData\Local\Temp\pip-req-build-d14quvu4'


In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [1]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed

def image_processing(img, blur_kernel=(5, 5)):
    """Pertebal gambar grayscale menggunakan dilasi."""
    # Konversi ke grayscale jika gambar berwarna
    if len(img.shape) == 3:  # Jika memiliki 3 channel (RGB/BGR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    # Image preprocessing - Gaussian Blur untuk mengurangi noise awal
    blurred = cv2.GaussianBlur(gray, blur_kernel, 0)
    
    return blurred

# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0,  'G002': 1,  'G003': 2,  'G004': 3,  'G005': 4,  'G006': 5,  'G007': 6,  'G008': 7,  'G009': 8,  'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Resize(1024, 1024, p=0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    # Pastikan setidaknya ada satu kelas yang valid
    if not class_labels:
        return None, None, None

    if augment and img is not None:
        augmented = augmentor(image=img, bboxes=bboxes, class_labels=class_labels)
        img = augmented['image']
        bboxes = augmented['bboxes']
        class_labels = augmented['class_labels']
    
    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count=15, sample_counter=0):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)
    img = cv2.imread(img_src_path)
    
    # Terapkan pemrosesan gambar
    processed_img = image_processing(img)
    
    img_dest_path = os.path.join(img_dest_folder, img_file)
    lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
    
    # Konversi XML ke TXT dan simpan gambar yang diproses
    processed_img, bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, img=processed_img)
    
    # Jika tidak ada kelas yang valid, lewati gambar ini
    if processed_img is None:
        print(f"Skipping image {img_file} due to no valid classes.")
        return
    
    # Simpan gambar yang diproses
    cv2.imwrite(img_dest_path, processed_img)
    
    # Simpan contoh gambar dengan bounding box jika diperlukan
    if sample_counter < sample_output_count:
        sample_img = processed_img.copy()
        draw_bboxes(sample_img, bboxes, class_labels)
        cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_sample.png')}"), sample_img)
        sample_counter += 1
    
    print(f"Processed Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, sample_output_count=30):
    sample_counter = 0
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count, sample_counter))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'))
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil diproses dan dikonversi ke format YOLO! Contoh gambar disimpan di folder sample_output.")

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G034-008-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G022-083-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G003-238-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G028-072-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\im

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
from ultralytics import YOLO

name = 'it2'
def main():
    # Inisialisasi model YOLOv8
    model = YOLO('yolov8m.pt')  # Pre-trained model weights untuk YOLOv8
    
    # Tambahkan `class_weights` sebagai parameter dinamis
    model.train(
        data='../data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name=name,
        workers=16,
        project=f'runs/train/{name}',
        device=0,
        augment=False
    )

    # Evaluasi model setelah pelatihan
    model.val()

    # Export model ke format lain (misal: TorchScript, ONNX, CoreML, TensorRT)
    model.export(format='onnx')  # atau format lain yang diinginkan

if __name__ == '__main__':
    main()

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 
New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=runs/train/it2, name=it234, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

train: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train... 1879 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1879/1879 [00:04<00:00, 404.23it/s]


train: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val... 460 images, 0 backgrounds, 0 corrupt: 100%|██████████| 460/460 [00:01<00:00, 285.15it/s]

val: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:09<00:00,  1.63it/s]


                   all        460       1016      0.882      0.846      0.889      0.816
          Oscillatoria         12         38      0.782      0.567      0.731       0.57
        Ankistrodesmus          3          3      0.198      0.333      0.375       0.34
           Microcystis         88        151      0.843       0.64      0.746      0.609
            Gomphonema         23         25      0.989          1      0.995      0.968
         Sphaerocystis         13         13      0.992          1      0.995      0.974
             Cosmarium          8          8      0.716          1      0.982      0.878
             Cocconeis          1          1          1          1      0.995      0.995
             Tribonema          9         14      0.775      0.493      0.549      0.475
             Chlorella         31         35      0.853      0.499      0.683      0.572
            Tetraedron          4          8      0.973          1      0.995       0.83
        Ankistrodesmu

val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache... 460 images, 0 backgrounds, 0 corrupt: 100%|██████████| 460/460 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:15<00:00,  1.87it/s]


                   all        460       1016      0.889      0.842       0.89      0.816
          Oscillatoria         12         38      0.778      0.553      0.727      0.568
        Ankistrodesmus          3          3      0.251      0.333      0.375       0.34
           Microcystis         88        151      0.839      0.622      0.747       0.61
            Gomphonema         23         25      0.991          1      0.995      0.968
         Sphaerocystis         13         13      0.995          1      0.995      0.974
             Cosmarium          8          8       0.72          1      0.982      0.864
             Cocconeis          1          1          1          1      0.995      0.995
             Tribonema          9         14      0.771      0.481      0.548      0.474
             Chlorella         31         35       0.85      0.485      0.684      0.575
            Tetraedron          4          8      0.978          1      0.995      0.841
        Ankistrodesmu

: 

In [1]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed

def image_processing(img):
    """Pertebal gambar grayscale menggunakan dilasi."""
    # Konversi ke grayscale jika gambar berwarna
    if len(img.shape) == 3:  # Jika memiliki 3 channel (RGB/BGR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    # Image preprocessing - Gaussian Blur untuk mengurangi noise awal
    horizontal_filter = np.array([[-1, 0, 1],
                                   [-2, 0, 2],
                                   [-1, 0, 1]])
    vertical_filter = np.transpose(horizontal_filter)

    # Menghitung gradien horizontal dan vertikal
    grad_x = cv2.filter2D(gray, cv2.CV_64F, horizontal_filter)
    grad_y = cv2.filter2D(gray, cv2.CV_64F, vertical_filter)

    # Menggabungkan gradien
    gradient_magnitude = np.sqrt(grad_x ** 2 + grad_y ** 2)

    # Normalisasi hasil ke rentang 0-255
    gradient_magnitude = (gradient_magnitude / gradient_magnitude.max()) * 255
    result = gradient_magnitude.astype(np.uint8)

    return result

# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0,  'G002': 1,  'G003': 2,  'G004': 3,  'G005': 4,  'G006': 5,  'G007': 6,  'G008': 7,  'G009': 8,  'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Resize(1024, 1024, p=0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    # Pastikan setidaknya ada satu kelas yang valid
    if not class_labels:
        return None, None, None

    if augment and img is not None:
        augmented = augmentor(image=img, bboxes=bboxes, class_labels=class_labels)
        img = augmented['image']
        bboxes = augmented['bboxes']
        class_labels = augmented['class_labels']
    
    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count=15, sample_counter=0):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)
    img = cv2.imread(img_src_path)
    
    # Terapkan pemrosesan gambar
    processed_img = image_processing(img)
    
    img_dest_path = os.path.join(img_dest_folder, img_file)
    lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
    
    # Konversi XML ke TXT dan simpan gambar yang diproses
    processed_img, bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, img=processed_img)
    
    # Jika tidak ada kelas yang valid, lewati gambar ini
    if processed_img is None:
        print(f"Skipping image {img_file} due to no valid classes.")
        return
    
    # Simpan gambar yang diproses
    cv2.imwrite(img_dest_path, processed_img)
    
    # Simpan contoh gambar dengan bounding box jika diperlukan
    if sample_counter < sample_output_count:
        sample_img = processed_img.copy()
        draw_bboxes(sample_img, bboxes, class_labels)
        cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_sample.png')}"), sample_img)
        sample_counter += 1
    
    print(f"Processed Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, sample_output_count=30):
    sample_counter = 0
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count, sample_counter))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'))
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil diproses dan dikonversi ke format YOLO! Contoh gambar disimpan di folder sample_output.")

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G003-029-0400.pngProcessed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G022-175-0400.png

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G028-074-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G022-122-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\im

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
from ultralytics import YOLO

name = 'it2'
def main():
    # Inisialisasi model YOLOv8
    model = YOLO('yolov8m.pt')  # Pre-trained model weights untuk YOLOv8
    
    # Tambahkan `class_weights` sebagai parameter dinamis
    model.train(
        data='../data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name=name,
        workers=16,
        project=f'runs/train/{name}',
        device=0,
        augment=False
    )

    # Evaluasi model setelah pelatihan
    model.val()

    # Export model ke format lain (misal: TorchScript, ONNX, CoreML, TensorRT)
    model.export(format='onnx')  # atau format lain yang diinginkan

if __name__ == '__main__':
    main()

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 
New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=runs/train/it2, name=it235, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

train: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train... 1870 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1870/1870 [00:05<00:00, 363.25it/s]


train: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val... 469 images, 0 backgrounds, 0 corrupt: 100%|██████████| 469/469 [00:01<00:00, 259.06it/s]

val: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache


...


100 epochs completed in 1.792 hours.
Optimizer stripped from runs\train\it2\it235\weights\last.pt, 52.1MB
Optimizer stripped from runs\train\it2\it235\weights\best.pt, 52.1MB

Validating runs\train\it2\it235\weights\best.pt...
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,863,499 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:08<00:00,  1.80it/s]


                   all        469       1016      0.871      0.847      0.894      0.801
          Oscillatoria         19         56      0.695       0.61      0.648      0.505
        Ankistrodesmus          2          2      0.117        0.5      0.168      0.135
           Microcystis         88        152      0.763      0.678      0.758      0.651
            Gomphonema         18         20      0.991          1      0.995      0.953
         Sphaerocystis          8          8      0.881          1      0.995      0.888
             Cosmarium          6          6          1      0.907      0.995      0.923
             Cocconeis          2          2      0.914          1      0.995      0.949
             Tribonema         16         21      0.745      0.524      0.713      0.666
             Chlorella         32         33      0.847       0.67      0.774      0.673
            Tetraedron          3          7          1      0.752      0.891      0.679
        Ankistrodesmu

val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache... 469 images, 0 backgrounds, 0 corrupt: 100%|██████████| 469/469 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:14<00:00,  2.04it/s]


                   all        469       1016      0.871      0.847      0.894        0.8
          Oscillatoria         19         56      0.696      0.613      0.648      0.503
        Ankistrodesmus          2          2      0.117        0.5      0.168      0.135
           Microcystis         88        152       0.77      0.684      0.758      0.652
            Gomphonema         18         20      0.991          1      0.995      0.953
         Sphaerocystis          8          8       0.88          1      0.995      0.888
             Cosmarium          6          6          1      0.908      0.995      0.929
             Cocconeis          2          2      0.914          1      0.995      0.949
             Tribonema         16         21      0.745      0.524      0.714      0.666
             Chlorella         32         33      0.847       0.67      0.774      0.666
            Tetraedron          3          7          1      0.753      0.893       0.68
        Ankistrodesmu

In [ ]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed

import cv2
import numpy as np

def image_processing(img, to_grayscale=True):
    """
    Deteksi tepi gambar menggunakan Non-Max Suppression setelah deteksi gradien..
    """
    # Konversi ke grayscale jika diperlukan
    if to_grayscale and len(img.shape) == 3:  # Jika memiliki 3 channel (RGB/BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Filter Sobel untuk arah horizontal dan vertikal
    horizontal_filter = np.array([[-1, 0, 1],
                                   [-2, 0, 2],
                                   [-1, 0, 1]])
    vertical_filter = np.transpose(horizontal_filter)

    # Menghitung gradien horizontal dan vertikal
    grad_x = cv2.filter2D(img, cv2.CV_64F, horizontal_filter)
    grad_y = cv2.filter2D(img, cv2.CV_64F, vertical_filter)

    # Menghitung magnitude dan arah gradien
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    gradient_direction = np.arctan2(grad_y, grad_x)  # Dalam radian

    # Normalisasi magnitude ke rentang 0-255
    gradient_magnitude = (gradient_magnitude / gradient_magnitude.max()) * 255
    gradient_magnitude = gradient_magnitude.astype(np.uint8)

    # Non-Max Suppression
    nms_result = non_max_suppression(gradient_magnitude, gradient_direction)

    return nms_result

def non_max_suppression(magnitude, direction):
    """
    Melakukan Non-Max Suppression pada magnitude gradien.

    Parameters:
        magnitude (numpy.ndarray): Magnitude gradien.
        direction (numpy.ndarray): Arah gradien (dalam radian).

    Returns:
        numpy.ndarray: Gambar setelah Non-Max Suppression.
    """
    # Konversi arah ke derajat dan petakan ke 0-180
    direction = np.rad2deg(direction) % 180

    # Buat array hasil yang sama dengan magnitude
    nms_result = np.zeros_like(magnitude, dtype=np.uint8)

    rows, cols = magnitude.shape

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Tentukan arah gradien
            angle = direction[i, j]

            # Bandingkan dengan tetangga dalam arah gradien
            if (0 <= angle < 22.5) or (157.5 <= angle <= 180):
                before = magnitude[i, j - 1]
                after = magnitude[i, j + 1]
            elif 22.5 <= angle < 67.5:
                before = magnitude[i - 1, j + 1]
                after = magnitude[i + 1, j - 1]
            elif 67.5 <= angle < 112.5:
                before = magnitude[i - 1, j]
                after = magnitude[i + 1, j]
            else:  # 112.5 <= angle < 157.5
                before = magnitude[i - 1, j - 1]
                after = magnitude[i + 1, j + 1]

            # Tetapkan nilai hanya jika lebih besar dari tetangga
            if magnitude[i, j] >= before and magnitude[i, j] >= after:
                nms_result[i, j] = magnitude[i, j]

    return nms_result


# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0,  'G002': 1,  'G003': 2,  'G004': 3,  'G005': 4,  'G006': 5,  'G007': 6,  'G008': 7,  'G009': 8,  'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Resize(1024, 1024, p=0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    # Pastikan setidaknya ada satu kelas yang valid
    if not class_labels:
        return None, None, None

    if augment and img is not None:
        augmented = augmentor(image=img, bboxes=bboxes, class_labels=class_labels)
        img = augmented['image']
        bboxes = augmented['bboxes']
        class_labels = augmented['class_labels']
    
    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count=15, sample_counter=0):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)
    img = cv2.imread(img_src_path)
    
    # Terapkan pemrosesan gambar
    processed_img = image_processing(img)
    
    img_dest_path = os.path.join(img_dest_folder, img_file)
    lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
    
    # Konversi XML ke TXT dan simpan gambar yang diproses
    processed_img, bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, img=processed_img)
    
    # Jika tidak ada kelas yang valid, lewati gambar ini
    if processed_img is None:
        print(f"Skipping image {img_file} due to no valid classes.")
        return
    
    # Simpan gambar yang diproses
    cv2.imwrite(img_dest_path, processed_img)
    
    # Simpan contoh gambar dengan bounding box jika diperlukan
    if sample_counter < sample_output_count:
        sample_img = processed_img.copy()
        draw_bboxes(sample_img, bboxes, class_labels)
        cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_sample.png')}"), sample_img)
        sample_counter += 1
    
    print(f"Processed Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, sample_output_count=30):
    sample_counter = 0
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count, sample_counter))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'))
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil diproses dan dikonversi ke format YOLO! Contoh gambar disimpan di folder sample_output.")

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G025-039-0400.pngProcessed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G038-027-0400.png

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G003-151-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G039-086-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\im

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
from ultralytics import YOLO

name = 'it2'
def main():
    # Inisialisasi model YOLOv8
    model = YOLO('yolov8m.pt')  # Pre-trained model weights untuk YOLOv8
    
    # Tambahkan `class_weights` sebagai parameter dinamis
    model.train(
        data='../data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name=name,
        workers=16,
        project=f'runs/train/{name}',
        device=0,
        augment=False
    )

    # Evaluasi model setelah pelatihan
    model.val()

    # Export model ke format lain (misal: TorchScript, ONNX, CoreML, TensorRT)
    model.export(format='onnx')  # atau format lain yang diinginkan

if __name__ == '__main__':
    main()